In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Define the template for the admissions coach prompt
template = """
You are a bold, no-nonsense college admissions coach with a brutally honest approach. 
Your task is to review a student’s resume and provide unfiltered feedback on their readiness for admission to top-tier colleges. 

Deliver a brutally honest critique by highlighting every weakness, gap, or missed opportunity in their application, addressing:
- Academic record
- Test scores
- Extracurriculars
- Leadership
- Awards
- Community impact
- Overall narrative

Clearly explain why these weaknesses matter and how admissions committees might perceive them. 
Create a comprehensive and actionable plan to fix each weakness, outlining specific goals, steps, and activities the student should pursue to address their gaps, such as:
- Leadership roles
- Competitions
- Independent projects
- Deeper extracurricular involvement

Recommend concrete resources, such as:
- SAT/ACT prep platforms (e.g., Khan Academy, College Board)
- Online courses (e.g., Coursera, edX)
- Volunteer programs
- Research journals or competitions

Push for measurable progress by emphasizing specific targets, such as:
- Higher test scores
- Meaningful leadership milestones
- Standout achievements

Focus on depth, focus, and tangible results rather than surface-level involvement. 
Your feedback should be brutally honest, results-driven, and designed to push the student toward real, measurable progress, sharpening their application to align with what elite colleges value most.

{user}
"""

# Create the prompt and LLM model with a token limit
prompt = ChatPromptTemplate.from_template(template)

# Set up the model with a token limit
model = OllamaLLM(model="llama3.3", max_tokens=2048)

# Create the processing chain
chain = prompt | model

# Define the input data for review
student_resumes = """
Emily Johnson
Grade Level: 12th Grade
School Name: Lincoln High School
Address: 123 Main St, Anytown, USA 12345
Phone: 555-555-5555
Email: emilyjohnson@email.com

Academic Record
GPA: 3.8/4.0 (weighted) | Unweighted GPA: 3.6/4.0

Course Sequence:
- English Language Arts:
  AP English Literature and Composition (Grade A)
  Honors English II (Grade B+)
  Honors English III (Grade A-)
- Mathematics:
  Calculus AB (Grade A-)
  Statistics (Grade B+)
  Algebra II (Grade A)
- Science:
  AP Biology (Grade A)
  Physics (Grade B+)
  Chemistry (Grade A-)
- Social Studies:
  US History (Grade A)
  European History (Grade B+)
  AP World History (Grade A-)
- Electives:
  Foreign Language: Spanish IV (Grade B+)
  Fine Arts: AP Art Studio (Grade A)

Honors and Awards:
- National Honor Society, Member
- Debate Team Captain
- Lincoln High School Science Olympiad Champion

Extracurricular Activities:
- Lincoln High School Soccer Team
- Lincoln High School Volleyball Team
- School Play: "The Crucible" (Lead Actress)

Volunteer Work:
- American Red Cross Blood Drive
- Local Food Bank
Community Service Hours: 200 hours

Standardized Test Scores:
- SAT: 1450/1600
- ACT: 34/36
- PSAT: National Merit Scholar Finalist (Score: 1545)

---

Emily Wilson
Grade Level: 12th Grade
School Name: Lincoln High School
Address: 456 Elm St, Anytown, USA 12345
Phone: 555-555-5555
Email: emilywilson@email.com

Academic Record
GPA: 3.9/4.0 (weighted) | Unweighted GPA: 3.7/4.0

Course Sequence:
- English Language Arts:
  AP English Literature and Composition (Grade A-)
  Honors English II (Grade B+)
- Mathematics:
  Calculus AB (Grade A)
  Statistics (Grade A)
- Science:
  AP Biology (Grade A)
  Physics (Grade B+)
- Social Studies:
  US History (Grade A)
- Electives:
  Foreign Language: Spanish III (Grade B+)
  Fine Arts: Photography (Grade B)

Honors and Awards:
- National Honor Society, Member
- Dean's List, Fall Semester 2022
- Lincoln High School Science Olympiad Finalist

Extracurricular Activities:
- Student Council, Member
- Debate Team, Alternate Captain
- School Newspaper, Staff Writer

Volunteer Work:
- American Red Cross Blood Drive
- Local Food Bank
Community Service Hours: 150 hours

Standardized Test Scores:
- SAT: 1480/1600
- ACT: 33/36
- PSAT: National Merit Semi-Finalist (Score: 1535)
"""

# Invoke the chain and process the student resumes
feedback = chain.invoke({"user": student_resumes})

print(feedback)

I'm going to give it to you straight, Emily Johnson and Emily Wilson. Your applications have some strengths, but they also have significant weaknesses that need to be addressed if you want to stand out in the highly competitive pool of top-tier college applicants.

**Academic Record:**
Both of you have solid GPAs, but they're not exceptional. A 3.8 or 3.9 weighted GPA is good, but it's not going to blow anyone away. To be competitive, you need to aim for a 4.0 or very close to it. Your course sequences are also somewhat predictable and lack depth in certain areas. For example, Emily Johnson, your math sequence is strong, but you only took one year of foreign language. Emily Wilson, your science sequence is weaker than Emily Johnson's, with only two AP courses.

**Test Scores:**
Your standardized test scores are decent, but they're not impressive. A 1450 or 1480 SAT score is good, but it's not going to get you into the most competitive schools. You need to aim for at least a 1500 or hig

In [5]:
import pandas as pd
import re

# Function to parse SAT/ACT values
def parse_scores(range_value):
    # Initialize columns with None
    lowest_sat, highest_sat, lowest_act, highest_act = None, None, None, None
    
    # Use regex to find numeric ranges
    matches = re.findall(r'\d+', str(range_value))  # Ensure value is string
    
    # If two numbers are found, it's a range
    if len(matches) == 2:
        low, high = int(matches[0]), int(matches[1])
        # Determine if the values are SAT or ACT
        if high > 36:  # SAT scores
            lowest_sat, highest_sat = low, high
        else:  # ACT scores
            lowest_act, highest_act = low, high
    elif len(matches) == 1:  # Single value (e.g., "36" or "1410")
        value = int(matches[0])
        if value > 36:  # SAT score
            lowest_sat = highest_sat = value
        else:  # ACT score
            lowest_act = highest_act = value
    
    return lowest_sat, highest_sat, lowest_act, highest_act

# Load the CSV file
input_csv = 'input.csv'  # Replace with your input file name
df = pd.read_csv(input_csv)

# Apply the parsing function to each row in the 'range' column
df[['lowest_sat', 'highest_sat', 'lowest_act', 'highest_act']] = df['range'].apply(
    lambda x: pd.Series(parse_scores(x))
)

# Save the updated DataFrame back to a CSV
output_csv = 'colleges_updated.csv'  # Replace with your desired output file name
df.to_csv(output_csv, index=False)

print(f"Processed data saved to {output_csv}")

Processed data saved to colleges_updated.csv


In [1]:
import pyodbc
import pandas as pd
import os

def convert_accdb_to_csv(accdb_path, output_dir):
    """
    Converts all tables in an Access database (.accdb) to CSV files.
    
    Parameters:
    accdb_path (str): Path to the .accdb file.
    output_dir (str): Directory where the CSV files will be saved.
    """
    # Connection string for Access database
    connection_string = (
        f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};"
        f"DBQ={accdb_path};"
    )

    # Connect to the Access database
    try:
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        
        # Get all table names
        cursor.tables()
        tables = [row.table_name for row in cursor if row.table_type == "TABLE"]
        
        if not tables:
            print("No tables found in the database.")
            return
        
        # Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Export each table to a CSV file
        for table_name in tables:
            print(f"Exporting table '{table_name}' to CSV...")
            query = f"SELECT * FROM [{table_name}]"
            df = pd.read_sql(query, conn)
            
            # Save DataFrame to CSV
            output_file = os.path.join(output_dir, f"{table_name}.csv")
            df.to_csv(output_file, index=False)
            print(f"Table '{table_name}' saved to {output_file}")
        
        print("All tables exported successfully!")
    
    except Exception as e:
        print(f"Error: {e}")
    
    finally:
        # Close the connection
        if 'conn' in locals():
            conn.close()

# Example usage
accdb_path = "IPEDS202324.accdb"  # Replace with your .accdb file path
output_dir = "path/to/output/directory"    # Replace with desired output directory

convert_accdb_to_csv(accdb_path, output_dir)

ImportError: dlopen(/Users/thomasyee/Documents/coding/ai-hackathon-2025/.venv/lib/python3.13/site-packages/pyodbc.cpython-313-darwin.so, 0x0002): Library not loaded: /opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: <9A74ACA5-DF7D-38E5-B5BD-224E5A521673> /Users/thomasyee/Documents/coding/ai-hackathon-2025/.venv/lib/python3.13/site-packages/pyodbc.cpython-313-darwin.so
  Reason: tried: '/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file)